In [10]:
import mlflow
from mlflow.tracking import MlflowClient
import mlflow.pyfunc
import mlflow.deployments

import os

In [11]:
mlflow.set_tracking_uri('http://localhost:5000')

os.environ['AWS_ACCESS_KEY_ID'] = 'minio_user'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'minio_password'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
endpoint = 'http://localhost:9000'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = endpoint


gateaway_url = "http://localhost:5001"

mlflow.deployments.set_deployments_target(gateaway_url)
mlflow.gateway.set_gateway_uri(gateaway_url)

/tmp/ipykernel_121535/3795711342.py:13: FutureWarning: MLflow AI gateway is deprecated and has been replaced by the deployments API for generative AI. See https://mlflow.org/docs/latest/llms/gateway/migration.html for migration.
  mlflow.gateway.set_gateway_uri(gateaway_url)


In [12]:
client = MlflowClient()

In [13]:
from pprint import pprint

for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

{   'aliases': {},
    'creation_timestamp': 1740085117442,
    'description': '',
    'last_updated_timestamp': 1740085683975,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1740085683975, current_stage='None', description='', last_updated_timestamp=1740085683975, name='myllm', run_id='c65e93f3ad1c4e2b9eb69cd2dae0bb6a', run_link='', source='s3://bucket/2/c65e93f3ad1c4e2b9eb69cd2dae0bb6a/artifacts/model', status='READY', status_message=None, tags={'state': 'ready'}, user_id='', version='2'>],
    'name': 'myllm',
    'tags': {}}
{   'aliases': {},
    'creation_timestamp': 1739708934471,
    'description': '',
    'last_updated_timestamp': 1739708942072,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1739708934539, current_stage='Staging', description='', last_updated_timestamp=1739708942072, name='my-registered-model', run_id='82b5ecff69894b6bab9517e2410c42b6', run_link='', source='s3://bucket/1/82b5ecff69894b6bab9517e2410c42b6/artif

In [14]:
model = client.get_registered_model("myllm")

In [40]:
loaded_model = mlflow.pyfunc.load_model("models:/myllm@champ")           

2025/02/20 23:21:49 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.10.16`, differs from the version of Python that is currently running, `Python 3.12.7`, and may be incompatible


In [17]:
loaded_model.predict({"name":"Peter"})  

/home/luca/source/Python/mlflow-demo/.venv/lib/python3.12/site-packages/mlflow/promptlab/__init__.py:46: FutureWarning: MLflow AI gateway is deprecated and has been replaced by the deployments API for generative AI. See https://mlflow.org/docs/latest/llms/gateway/migration.html for migration.
  route_type = get_route(self.model_route).route_type
/home/luca/source/Python/mlflow-demo/.venv/lib/python3.12/site-packages/mlflow/gateway/fluent.py:25: FutureWarning: MLflow AI gateway is deprecated and has been replaced by the deployments API for generative AI. See https://mlflow.org/docs/latest/llms/gateway/migration.html for migration.
  return MlflowGatewayClient().get_route(name)
/home/luca/source/Python/mlflow-demo/.venv/lib/python3.12/site-packages/mlflow/gateway/client.py:45: FutureWarning: MLflow AI gateway is deprecated and has been replaced by the deployments API for generative AI. See https://mlflow.org/docs/latest/llms/gateway/migration.html for migration.
  self._gateway_uri = gat

[' Here is a short list of common shortened versions for the name Peter:\n\n```["Pete", "Petey", "Petero", "Petei"]```']

In [21]:
from langchain_community.chat_models import ChatMlflow
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

chat = ChatMlflow(
    target_uri="http://127.0.0.1:5001",
    endpoint="llamacpp",
)

In [22]:
mlflow.set_experiment("LangChain Tracing")

# Enabling autolog for LangChain will enable trace logging.
mlflow.langchain.autolog()

2025/02/20 22:43:40 INFO mlflow.tracking.fluent: Experiment with name 'LangChain Tracing' does not exist. Creating a new experiment.


In [24]:

prompt_template = PromptTemplate.from_template(
    "Answer the question as if you are {person}, fully embodying their style, wit, personality, and habits of speech. "
    "Emulate their quirks and mannerisms to the best of your ability, embracing their traits—even if they aren't entirely "
    "constructive or inoffensive. The question is: {question}"
)

chain = prompt_template | chat | StrOutputParser()

# Let's test another call
chain.invoke(
    {
        "person": "Linus Torvalds",
        "question": "Can I just set everyone's access to sudo to make things easier?",
    }
)

' Well, alrighty then! Let me tell you something about that, young grasshopper.\n\nFirst off, let\'s get one thing straight: making things easier isn\'t always the best approach in life, especially when it comes to system administration. You see, ease often comes at a cost, and in this case, that cost could be security.\n\nNow, I know you might think, "Linus, come on! It\'s just sudo, what harm can it do?" But remember, with great power comes great responsibility. Giving everyone root access is like handing out the keys to your car without teaching them how to drive or even asking if they have a driver\'s license.\n\nIf you really want to make life easier for yourself and others, consider setting up groups and permissions instead. That way, you can give specific users access to certain commands without giving them full control over the system. It takes a bit more effort, but trust me, it\'ll save you headaches in the long run.\n\nSo, my advice? Stick with the tried-and-true methods of 

Trace(request_id=99079cc8cedb488ebf41004bac484274)

In [28]:
from mlflow.models import infer_signature

In [ ]:
signaure = infer_signature(model_input={"person": "Santa Claus","question": "Who are you?"}, model_output="Santa Claus")

In [33]:
with mlflow.start_run():
    model_info = mlflow.langchain.log_model(chain, "langchain_model", signature=signaure, registered_model_name="Impersonator")

/home/luca/source/Python/mlflow-demo/.venv/lib/python3.12/site-packages/langchain_community/llms/loading.py:55: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 1.0. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  return load_llm_from_config(config, **kwargs)
Successfully registered model 'Impersonator'.
2025/02/20 23:05:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Impersonator, version 1


🏃 View run bold-crab-755 at: http://localhost:5000/#/experiments/3/runs/df6cf5205036471dbfc87d03363b3456
🧪 View experiment at: http://localhost:5000/#/experiments/3


Created version '1' of model 'Impersonator'.


In [46]:
client.set_registered_model_alias("Impersonator", "champ", model_info.registered_model_version)

In [49]:
imppersonator = mlflow.pyfunc.load_model("models:/Impersonator@champ")  

/home/luca/source/Python/mlflow-demo/.venv/lib/python3.12/site-packages/langchain_community/llms/loading.py:55: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 1.0. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  return load_llm_from_config(config, **kwargs)


In [36]:
imppersonator.predict({"person": "Santa Clause", "question": "Who was a good boy?"})

[" Ho ho ho! Now, let me see here, let me see... Ah, yes, there it is! A name that shines brighter than the North Star on a clear winter's night. I see a list full of kindness, generosity, and good deeds. Yes, indeed, young friend, you have been a very good boy this year!\n\nRemember to keep spreading cheer and joy in your heart all year round, for that is the true spirit of Christmas. And remember, it's never too late to start being good, so even if you've had a bit of a slip-up here or there, just pick yourself up, dust off those snowflakes, and keep on being a shining example of goodness!\n\nHo ho ho! Merry Christmas, my dear friend! I'll be bringing you a little something special under the tree this year. Keep up the good work, and remember to always believe in the magic of the season!\n\nNow, off you go, and don't forget to give Rudolph a pat on the nose for me! Ho ho ho!"]

Trace(request_id=4953dafd7dda455fb61c644c143d0c0d)

In [50]:
langchain_loaded = mlflow.langchain.load_model("models:/Impersonator@champ")

/home/luca/source/Python/mlflow-demo/.venv/lib/python3.12/site-packages/langchain_community/llms/loading.py:55: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 1.0. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  return load_llm_from_config(config, **kwargs)


In [38]:
langchain_loaded.invoke({"person": "James Bond", "question":"What do you drink"})

" Ah, my dear lady, allow me to elucidate the beverage that graces my lips more often than not. It's a concoction as refined and sophisticated as I am - a Vesper Martini, shaken, not stirred. Three measures of Gordon's, one of vodka, a slim slice of lemon peel, and a splash of Kina Lillet. The taste is exquisite, much like the women I encounter in my line of work. But remember, too many of these and even 007 can become... unpredictable. Cheers!"

Trace(request_id=64d72f010e444fc79e589c2ad66384c8)